<a href="https://colab.research.google.com/github/sudo-ken/Torrents-to-Google-Drive/blob/master/Torrents_to_Google_Drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **<font color='blue'> Torrents to Google Drive - Shared by [SudoKen](https://www.youtube.com/channel/UCZq5YhZIR-250zX_MzkTbxA?sub_confirmation=1) </font>**


---
#Mount Google Drive with Rclone 

In [0]:
#@markdown <h3>📝 Note: Run this before using Rclone.</h3>
#@markdown <h4>📝 Upload an rclone profile</4>
Setup_Time_Zone = False 

import os; from google.colab import files; from IPython.display import HTML, clear_output

def upload_conf():
  try:
    display(HTML("<h2 style=\"font-family:Trebuchet MS;color:#446785;\">Please upload the config file of rclone (rclone.conf) from your computer.</h2><br>"))
    UploadConfig = files.upload().keys()
    clear_output(wait=True)
    if len(UploadConfig) == 0:
      return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">File upload has been cancelled during upload file.</h2><br></center>"))
    elif len(UploadConfig) == 1:
      for fn in UploadConfig:
        if os.path.isfile("/content/" + fn) == True:
          os.environ["rclone_conf"] = fn
          !mv -f "$rclone_conf" /root/.rclone.conf
          !chmod 666 /root/.rclone.conf
          if Setup_Time_Zone == True:
            !sudo dpkg-reconfigure tzdata
            clear_output(wait=True)
          if os.path.isfile("/usr/bin/rclone") == True:
            return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#00b24c;\">Config has been changed.</h2><br></center>"))
          else:
            !rm -rf /content/sample_data/
            !curl -s https://rclone.org/install.sh | sudo bash
            clear_output(wait=True)
            return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#446785;\">Installation has been successfully completed.</h2><br></center>"))
        else:
          return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">File upload has been failed during upload file.</h2><br></center>"))
    else:
      for fn in UploadConfig:
        os.environ["rclone_conf"] = fn
        !rm -f "$rclone_conf"
      return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Please uploading only one file at a time.</h2><br></center>"))
  except:
    clear_output(wait=True)
    return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Error occurred during upload file.</h2><br></center>"))

upload_conf()

In [0]:
# ============================= FORM ============================= #
#@markdown <h3>📝 Run this to mount the drive with rclone</h3>
rclone_config_name = "YourDrive" #@param {type:"string"}
local_mount_location = "/content/udrive/" #@param {type:"string"}
#@markdown The default qBittorrent download location is '/content/udrive/' (they get downloaded to '/content/qbittorrent/' first)
# ============================= FORM ============================= #
import time
import os

# clear nohup
open("nohup.out", 'w').close()


# unmount first

!fusermount -u $local_mount_location 2>/dev/null


# mount without waiting for the command to complete
!mkdir $local_mount_location 2>/dev/null
!nohup rclone mount $rclone_config_name: $local_mount_location --buffer-size 96M & 
  


# Show the output that was written to nohup
time.sleep(3)
f = open(r"nohup.out", "r")
nohupText = f.read()
f.close()


dirs = os.listdir(local_mount_location)

if len(dirs) > 0:
  clear_output(wait=True)
  print("Succeeded. ", str(len(dirs)), "dirs found at", local_mount_location)

else:
  print("\n\nNot succeeded. No files or directories in mounted location. \nCheck your config name and content. If the rclone command was not found, run the cell above.\n\n")
  print("log:\n", nohupText)

---
# <img src='https://szyha.github.io/RcloneLabArchive/img/title_qbittorrent.png' height="45" alt="qBittorrent"/>

In [0]:
# ============================= FORM ============================= #
#@markdown <h3>📝CREATE CONFIG FILE</h3>
connections = 450 #@param {type:"integer"}
savePath = "/content/udrive/" #@param {type:"string"}
install_search_plugins = True #@param {type:'boolean'}
#https://github.com/thim0o/Google-Colab-Peerflix-Server
# ============================= FORM ============================= #

import os
import logging
import json
import requests
import ast

configPath = r"/root/.config/qBittorrent/qBittorrent.conf"


#CREATE THE CONFIG
data = r"""
[AutoRun][AutoRun]
enabled=true
program=chmod -R 666 \"%D\"

[BitTorrent]
Session\CreateTorrentSubfolder=true
Session\DisableAutoTMMByDefault=false
Session\DisableAutoTMMTriggers\CategoryChanged=false
Session\DisableAutoTMMTriggers\CategorySavePathChanged=false
Session\DisableAutoTMMTriggers\DefaultSavePathChanged=true
Session\GlobalMaxSeedingMinutes=1000

[Core]
AutoDeleteAddedTorrentFile=IfAdded

[Preferences]
Downloads\DiskWriteCacheSize=64
Session\SuggestMode=true
Advanced\AnnounceToAllTrackers=true
Session\AnnounceToAllTiers=true
Bittorrent\AddTrackers=true
Bittorrent\MaxConnecs={connections}
Bittorrent\MaxConnecsPerTorrent=-1
Bittorrent\MaxRatio=4
Bittorrent\MaxRatioAction=0
Bittorrent\MaxUploads=8
Bittorrent\PeX=true
Bittorrent\TrackersList=http://nyaa.tracker.wf:7777/announce\n\nhttp://sukebei.tracker.wf:8888/announce\n\nudp://tracker.coppersurfer.tk:6969/announce\n\nudp://tracker.open-internet.nl:6969/announce\n\nudp://tracker.leechers-paradise.org:6969/announce\n\nudp://tracker.internetwarriors.net:1337/announce\n\nudp://tracker.opentrackr.org:1337/announce\n\nudp://9.rarbg.to:2710/announce\n\nudp://9.rarbg.me:2710/announce\n\nhttp://tracker3.itzmx.com:6961/announce\n\nhttp://tracker1.itzmx.com:8080/announce\n\nudp://exodus.desync.com:6969/announce\n\nudp://explodie.org:6969/announce\n\nudp://ipv4.tracker.harry.lu:80/announce\n\nudp://denis.stalker.upeer.me:6969/announce\n\nudp://tracker.torrent.eu.org:451/announce\n\nudp://tracker.tiny-vps.com:6969/announce\n\nudp://thetracker.org:80/announce\n\nudp://open.demonii.si:1337/announce\n\nudp://tracker4.itzmx.com:2710/announce\n\nudp://tracker.cyberia.is:6969/announce\n\nudp://retracker.netbynet.ru:2710/announce\n\nudp://62.138.0.158:6969/announce\n\nudp://188.241.58.209:6969/announce\n\nudp://188.241.58.209:6969/announce\n\nudp://185.225.17.100:1337/announce\n\nudp://62.210.79.110:1337/announce\n\nudp://151.80.120.112:2710/announce\n\nudp://151.80.120.114:2710/announce\n\nhttp://176.113.71.19:6961/announce\n\nhttp://172.64.195.37:8080/announce\n\nudp://208.83.20.20:6969/announce\n\nudp://184.105.151.164:6969/announce\n\nudp://51.15.40.114:80/announce\n\nudp://5.2.79.22:6969/announce\n\nudp://91.216.110.52:451/announce\n\nudp://5.206.58.23:6969/announce\n\nudp://176.31.106.35:80/announce\n\nudp://5.2.79.219:1337/announce\n\nudp://212.1.226.176:2710/announce\n\nudp://159.100.245.181:6969/announce\n\nudp://212.1.226.176:2710/announce\n
Bittorrent\uTP_rate_limited=false
Connection\GlobalDLLimitAlt=0
Connection\GlobalUPLimit=30000
Connection\GlobalUPLimitAlt=0
Connection\PortRangeMin=12121
Downloads\PreAllocation=false
Downloads\SavePath={savePath}
Downloads\ScanDirsV2=@Variant(\0\0\0\x1c\0\0\0\x1\0\0\0\x12\0/\0\x63\0o\0n\0t\0\x65\0n\0t\0/\0\0\0\x2\0\0\0\x1)
Downloads\StartInPause=false
Downloads\TempPath=/content/qBittorrent/
Downloads\TempPathEnabled=true
General\Locale=en
General\UseRandomPort=false
Queueing\IgnoreSlowTorrents=true
Queueing\MaxActiveDownloads=8
Queueing\MaxActiveTorrents=15
Queueing\MaxActiveUploads=8
Queueing\QueueingEnabled=true
WebUI\Address=*
WebUI\AlternativeUIEnabled=false
WebUI\AuthSubnetWhitelist=@Invalid()
WebUI\AuthSubnetWhitelistEnabled=false
WebUI\ClickjackingProtection=true
WebUI\HTTPS\Enabled=false
WebUI\HostHeaderValidation=true
WebUI\LocalHostAuth=false
WebUI\Password_ha1=@ByteArray(322517f6a6fc65697957a1f9da700b43)
WebUI\Port=6006
WebUI\RootFolder=
WebUI\Username=rclonelab
""".format(connections=connections,
          savePath=savePath,
          ).split("\n")




# SAVE THE CONFIG
!mkdir /root/.config/qBittorrent/
open(configPath, 'w').close()

configFile = open(configPath, "a+")

for line in data:
  configFile.write(line + "\n")
  
configFile.close()


# PRINT THE CONFIG FOR DEBUGGING
f = open(configPath, "r")
text = f.read()
f.close()

clear_output(wait=True)
print("created the config successfully")

f = open(r"nohup.out", "w+").close()
# Download search engines

engineDirectory = r"/root/.local/share/data/qBittorrent/nova3/engines/"
if install_search_plugins:
  
  !mkdir $engineDirectory >/dev/null
  pluginsInstalled = 0
  searchEngineLinks = [r"https://raw.githubusercontent.com/MaurizioRicci/qBittorrent_search_engine/master/kickass_torrent.py",
                       r"https://raw.githubusercontent.com/MaurizioRicci/qBittorrent_search_engine/master/extratorrent.py",
                       r"https://raw.githubusercontent.com/MaurizioRicci/qBittorrent_search_engine/master/ettv.py",
                       r"https://raw.githubusercontent.com/hannsen/qbittorrent_search_plugins/master/demonoid.py",
                       r"https://raw.githubusercontent.com/MaurizioRicci/qBittorrent_search_engine/master/torrentproject.py",
                       r"https://raw.githubusercontent.com/Pireo/hello-world/master/rutor.py",
                       r"https://raw.githubusercontent.com/nindogo/qbtSearchScripts/master/magnetdl.py",
                       r"https://github.com/qbittorrent/search-plugins/blob/master/nova3/engines/zooqle.py",
                       r"https://github.com/qbittorrent/search-plugins/blob/master/nova3/engines/rarbg.py",
                       r"https://github.com/qbittorrent/search-plugins/blob/master/nova3/engines/torlock.py",
                       r"https://github.com/qbittorrent/search-plugins/blob/master/nova3/engines/btdb.py",
                       
                        ]
  cmd = ""
  for link in searchEngineLinks:
    !nohup wget -nc -P /root/.local/share/data/qBittorrent/nova3/engines/ $link >/dev/null 2>&1 &
    pluginsInstalled += 1
    clear_output(wait=True)
  !$cmd

  print(pluginsInstalled, "plugins installed")

f = open(r"nohup.out", "r")
nohupText = f.read()
print(nohupText)

dirs = os.listdir(engineDirectory)
print(len(dirs), "in plugins folder")


In [0]:
# ============================= FORM ============================= #
#@markdown <h3>📝 INSTALL QBITTORRENT AND FILE MANAGER</h3>
Old_Version = False #@param {type:"boolean"}
Install_File_Manager = True #@param {type:"boolean"}
subdomain = "random" #@param {type:"string"}


# ================================================================ #

import os, psutil, IPython, uuid, time
import ipywidgets as widgets

from IPython.display import HTML, clear_output
from google.colab import output

#kill qbittorrent to be able to reload config
!pkill qbittorrent-nox
!pkill autossh
SuccessRun = widgets.Button(
    description='✔ Successfully',
    disabled=True,
    button_style='success'
)

UnsuccessfullyRun = widgets.Button(
    description='✘ Unsuccessfully',
    disabled=True,
    button_style='danger'
)

class MakeButton(object):
  def __init__(self, title, callback):
    self._title = title
    self._callback = callback
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    template = """<button class="p-Widget jupyter-widgets jupyter-button widget-button mod-info" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id)
    return html

def RandomGenerator():
  return time.strftime("%S") + str(time.time()).split(".")[-1]

def CheckProcess(process, command):
  for pid in psutil.pids():
    try:
      p = psutil.Process(pid)
      if process in p.name():
        for arg in p.cmdline():
          if command in str(arg):  
            return True
          else:
            pass
      else:
        pass
    except:
      continue
  
def AutoSSH(name,port):
  get_ipython().system_raw("autossh -l " + name + " -M 0 -fNT -o 'StrictHostKeyChecking=no' -o 'ServerAliveInterval 300' -o 'ServerAliveCountMax 30' -R 80:localhost:" + port + " ssh.localhost.run &")
  get_ipython().system_raw("autossh -M 0 -fNT -o 'StrictHostKeyChecking=no' -o 'ServerAliveInterval 300' -o 'ServerAliveCountMax 30' -R " + name + ":80:localhost:" + port + " serveo.net &")

def Start_Server():
  if CheckProcess("qbittorrent-nox", "") != True:
    print("Starting qbittorrent...")
    !qbittorrent-nox -d --webui-port=6006
  if CheckProcess("autossh", Random_Number) != True:
      AutoSSH(Random_Number, "6006")
  if Install_File_Manager == True:
    if os.path.isfile("/tools/node/bin/cloudcmd") == False:
      clear_output(wait=True)
      !npm i -g npm
      !npm i cloudcmd -g
    if CheckProcess("cloudcmd", "") != True:
      get_ipython().system_raw("cloudcmd --online --no-auth --show-config --show-file-name --editor 'deepword' --packer 'tar' --port 7007 --progress --no-confirm-copy --confirm-move --name 'RcloneLab File Manager' --keys-panel --no-contact --console --sync-console-path --no-terminal --no-vim --columns 'name-size-date' --no-log &")
    if CheckProcess("autossh", "fm" + Random_Number) != True:
      AutoSSH("fm" + Random_Number, "7007")



try:
  if subdomain == "random":
    
    Random_NumberMT = RandomGenerator()
    Random_Number = RandomGenerator()
  else:
    Random_NumberMT = subdomain
    Random_Number = subdomain

#    !rm -rf /content/{sample_data,.config}/
    
  if os.path.isfile("/usr/bin/qbittorrent-nox") == False:
    !apt update -qq -y
    !yes "" | add-apt-repository ppa:qbittorrent-team/qbittorrent-stable
    if Old_Version == True:
      !apt install qbittorrent-nox=4.0.3-1 -qq -y
    else:
      !apt install qbittorrent-nox -qq -y
      
    clear_output(wait=True)

  if os.path.isfile("/usr/bin/autossh") == False:
    !apt install autossh -qq -y
  Start_Server()
  display(SuccessRun)
  display(MakeButton("Recheck", Start_Server))
  display(HTML("<h2 style=\"font-family:Trebuchet MS;color:#446785;\">qBittorrent</h2><h4 style=\"font-family:Trebuchet MS;color:#446785;\"><a style=\"font-family:Trebuchet MS;color:#356ebf;\" href=\"http://" + Random_Number + ".localhost.run\" target=\"_blank\">Website 1</a><br>" \
               "<a style=\"font-family:Trebuchet MS;color:#356ebf;\" href=\"http://" + Random_Number + ".serveo.net\" target=\"_blank\">Website 2</a></h4>"))
  if Install_File_Manager == True:
        display(HTML(
            "<h2 style=\"font-family:Trebuchet MS;color:#446785;\">File Manager</h2><h4 style=\"font-family:Trebuchet MS;color:#446785;\"><a style=\"font-family:Trebuchet MS;color:#356ebf;\" href=\"https://fm" + Random_Number + ".localhost.run/fs/content/\" target=\"_blank\">Website 1</a><br>"  "<a style=\"font-family:Trebuchet MS;color:#356ebf;\" href=\"https://fm" + Random_Number + ".serveo.net/fs/content/\" target=\"_blank\">Website</a></h3>"))
except Exception as e:
  console.log(e)
  display(UnsuccessfullyRun)

#◀️ To get more cool stuff like this, subscribe to the Youtube Channel: [SudoKen](https://www.youtube.com/channel/UCZq5YhZIR-250zX_MzkTbxA?sub_confirmation=1)